<a href="https://colab.research.google.com/github/SiriBatchu/CMPE-257/blob/main/Colab/Reading_and_HW_7_Feature_Importance_and_Amalgamation_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import the libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.decomposition import PCA

from tensorflow import keras
from tensorflow.keras import layers
import gc

### Upload the dataset

In [ ]:
file_path = '/content/HouseData_CA_Complete.csv'
data = pd.read_csv(file_path)

### Limit the dataset to a random sample of 1000 rows to avoid memory overload


In [ ]:
data = data.sample(n=1000, random_state=42)


### Missing value analysis

In [ ]:
missing_percentage = (data.isnull().sum() / len(data)) * 100
missing_percentage_sorted = missing_percentage.sort_values(ascending=False)
print("Missing Value Percentages:")
print(missing_percentage_sorted[missing_percentage_sorted > 0])

Missing Value Percentages:
builder_name        100.0
builder_id          100.0
half_baths           87.9
agent_nrds_id        85.4
broker_id            80.8
broker_name          78.5
parking_garage       77.6
unit                 76.3
hoa_fee              74.2
office_id            73.5
office_email         70.6
lot_sqft             67.7
agent_id             66.0
stories              62.4
agent_phones         62.0
office_phones        61.8
agent_email          61.4
office_mls_set       59.8
agent_name           59.7
agent_mls_set        59.7
office_name          59.7
list_price_max       56.8
list_price_min       56.6
year_built           55.5
last_sold_date       50.9
sold_price           50.9
price_per_sqft       49.4
sqft                 49.4
estimated_value      49.1
full_baths           44.4
list_price           43.7
beds                 43.7
assessed_value       42.7
text                 20.1
neighborhoods        19.0
primary_photo         6.6
alt_photos            6.6
longitude  

### Dropping duplicate records of the same property

In [ ]:
data = data.drop_duplicates(subset='property_id')
data = data.reset_index(drop=True)
data

,property_url,property_id,listing_id,mls,mls_id,status,text,style,full_street_line,street,...,builder_id,builder_name,office_id,office_mls_set,office_name,office_email,office_phones,nearby_schools,primary_photo,alt_photos
0,https://www.realtor.com/rentals/details/80824-...,1158980124,2967783803,AVAL,61090897,FOR_RENT,Beautiful Ryder 2 Floor Plan features 3 bedroo...,SINGLE_FAMILY,80824 Hermitage,80824 Hermitage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Coachella Valley Unified School District, Rive...",http://ap.rdcpix.com/f021c7a909c69949d368d00fd...,http://ap.rdcpix.com/f021c7a909c69949d368d00fd...
1,https://www.realtor.com/rentals/details/1601-N...,9142754349,2974996853,ZUMU,38977815,FOR_RENT,"Extension 2030 - This beautiful, sunny, light-...",APARTMENT,1601 Noe St,1601 Noe St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"San Francisco Unified School District, San Fra...",http://ap.rdcpix.com/2c8996e68c6ec75a52e6496b0...,http://ap.rdcpix.com/2c8996e68c6ec75a52e6496b0...
2,https://www.realtor.com/rentals/details/570-Al...,2242564289,2974926737,MRCA,LG24233049,FOR_RENT,Nestled within the exclusive gated community o...,SINGLE_FAMILY,570 Allview Ter,570 Allview Ter,...,NaN,NaN,256084.0,O-MRCA-LCNP,Coldwell Banker Realty - Newport Beach,900624.lead@leads.leadrouter.com,"[{'number': '9496441600', 'type': 'Office', 'p...",Laguna Beach Unified School District,http://ap.rdcpix.com/6fc02ad4056a0ddd6d8e730e8...,http://ap.rdcpix.com/6fc02ad4056a0ddd6d8e730e8...
3,https://www.realtor.com/rentals/details/11059-...,2254354449,2966780956,ZILL,43vhmf4dy544d,FOR_RENT,LUXURY APARTMENTS THAT SUIT YOUR LIFESTYLE Loc...,APARTMENT,11059 McCormick St,11059 McCormick St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles Unified School District,http://ar.rdcpix.com/a86f04bd532ff0ce43250358b...,http://ar.rdcpix.com/a86f04bd532ff0ce43250358b...
4,https://www.realtor.com/rentals/details/450-S-...,9608413152,2951082408,APTL,p31967036,FOR_RENT,Dare to have it all in a community centered ar...,APARTMENT,450 S La Fayette Park Pl,450 S La Fayette Park Pl,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles Unified School District,http://ar.rdcpix.com/1dee0ba6be9ad1938e786145f...,http://ar.rdcpix.com/1dee0ba6be9ad1938e786145f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,https://www.realtor.com/rentals/details/10034-...,9918669775,2974945837,MRCA,SR24233364,FOR_RENT,Quiet location next to 405 and 118 freeways. ....,CONDOS,10034 Noble Ave,10034 Noble Ave,...,NaN,NaN,1374117.0,O-MRCA-F3510002,Keller Williams Realty Calabasas,meny@kw.com,"[{'number': '8186576500', 'type': 'Office', 'p...",Los Angeles Unified School District,http://ap.rdcpix.com/f8063706dc0b03a0703aa839d...,http://ap.rdcpix.com/f8063706dc0b03a0703aa839d...
991,https://www.realtor.com/rentals/details/1709-S...,2577700605,2965319115,ZMPC,1331864,FOR_RENT,Finding an apartment home in the Los Angeles a...,APARTMENT,1709 S Westgate Ave,1709 S Westgate Ave,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Los Angeles Unified School District, New West ...",http://ar.rdcpix.com/388302aa4b5b80228beb45935...,http://ar.rdcpix.com/388302aa4b5b80228beb45935...
992,https://www.realtor.com/rentals/details/107-Da...,9766528025,2970632076,SDCA,240018933,FOR_RENT,3bed 4 bath plus bonus room Open floor plan to...,SINGLE_FAMILY,107 Daisy Ave Unit 3,107 Daisy Ave,...,NaN,NaN,NaN,O-SDCA-991611,Colwell Realty IB Inc,george@colwellrealtyib.com,"[{'number': '6194232444', 'type': None, 'prima...","Sweetwater Union High School District, South B...",http://ap.rdcpix.com/b6ea1578148e1c6d874ae4e67...,http://ap.rdcpix.com/b6ea1578148e1c6d874ae4e67...
993,https://www.realtor.com/rentals/details/21900-...,2932843267,2973816358,MRCA,SR24214665,FOR_RENT,Welcome to this beautifully remodeled townhous...,TOWNHOMES,21900 Marylee St Unit 290,21900 Marylee St,...,NaN,NaN,2890678.0,O-MRCA-F2918005,Rodeo Realty,kendavis@rodeore.com,"[{'number': '(818) 222-7707', 'type': 'Office'...",Los Angeles Unified School District,http://ap.rdcpix.com/e83

In [ ]:
data.describe()

,property_id,listing_id,zip_code,beds,full_baths,half_baths,sqft,year_built,days_on_mls,list_price,...,fips_code,stories,hoa_fee,parking_garage,agent_id,agent_nrds_id,broker_id,builder_id,builder_name,office_id
count,9.950000e+02,9.950000e+02,994.000000,561.000000,554.000000,120.000000,504.000000,443.000000,995.000000,561.000000,...,980.000000,374.000000,257.000000,224.000000,3.380000e+02,1.460000e+02,1.910000e+02,0.0,0.0,2.640000e+02
mean,4.627736e+09,2.966012e+09,92290.743461,2.538324,2.072202,1.058333,1890.077381,1984.498871,180.641206,8563.253119,...,6051.377551,2.139037,64.708171,1.982143,9.347623e+06,1.896037e+08,5.809952e+06,NaN,NaN,5.196358e+06
std,3.610005e+09,7.473225e+07,1765.685796,1.397335,1.385218,0.298338,2708.391508,27.049843,172.586207,29078.776769,...,23.083459,4.537236,298.559858,0.763308,2.575917e+07,3.902214e+07,1.713158e+07,NaN,NaN,1.689909e+07
min,1.002734e+09,6.213866e+08,90004.000000,0.000000,1.000000,1.000000,242.000000,1897.000000,11.000000,875.000000,...,6001.000000,1.000000,0.000000,1.000000,3.446000e+04,1.505889e+08,7.110170e+05,NaN,NaN,1.387000e+03
25%,1.718557e+09,2.966545e+09,90429.000000,2.000000,1.000000,1.000000,925.250000,1969.000000,43.000000,2600.000000,...,6037.000000,1.000000,0.000000,2.000000,1.153772e+06,1.755122e+08,1.679093e+06,NaN,NaN,8.588390e+05
50%,2.595606e+09,2.969173e+09,92125.000000,2.000000,2.000000,1.000000,1376.500000,1986.000000,153.000000,3750.000000,...,6053.000000,2.000000,0.000000,2.000000,1.949567e+06,1.860354e+08,3.090370e+06,NaN,NaN,2.788628e+06
75%,9.265716e+09,2.973941e+09,93559.500000,3.000000,3.000000,1.000000,2161.500000,2006.000000,233.000000,6500.000000,...,6071.500000,2.000000,0.000000,2.000000,3.349740e+06,2.055023e+08,4.299307e+06,NaN,NaN,3.374192e+06
max,9.998076e+09,2.975391e+09,96150.000000,8.000000,15.000000,3.000000,50177.000000,2024.000000,730.000000,585000.000000,...,6115.000000,61.000000,2963.000000,6.000000,1.005803e+08,5.735008e+08,1.005386e+08,NaN,NaN,1.004019e+08


In [ ]:
data.shape

(995, 57)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 57 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   property_url      995 non-null    object 
 1   property_id       995 non-null    int64  
 2   listing_id        995 non-null    int64  
 3   mls               995 non-null    object 
 4   mls_id            995 non-null    object 
 5   status            995 non-null    object 
 6   text              796 non-null    object 
 7   style             995 non-null    object 
 8   full_street_line  991 non-null    object 
 9   street            989 non-null    object 
 10  unit              236 non-null    object 
 11  city              994 non-null    object 
 12  state             995 non-null    object 
 13  zip_code          994 non-null    float64
 14  beds              561 non-null    float64
 15  full_baths        554 non-null    float64
 16  half_baths        120 non-null    float64
 1

In [ ]:
data.head(5)

,property_url,property_id,listing_id,mls,mls_id,status,text,style,full_street_line,street,...,builder_id,builder_name,office_id,office_mls_set,office_name,office_email,office_phones,nearby_schools,primary_photo,alt_photos
0,https://www.realtor.com/rentals/details/80824-...,1158980124,2967783803,AVAL,61090897,FOR_RENT,Beautiful Ryder 2 Floor Plan features 3 bedroo...,SINGLE_FAMILY,80824 Hermitage,80824 Hermitage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Coachella Valley Unified School District, Rive...",http://ap.rdcpix.com/f021c7a909c69949d368d00fd...,http://ap.rdcpix.com/f021c7a909c69949d368d00fd...
1,https://www.realtor.com/rentals/details/1601-N...,9142754349,2974996853,ZUMU,38977815,FOR_RENT,"Extension 2030 - This beautiful, sunny, light-...",APARTMENT,1601 Noe St,1601 Noe St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"San Francisco Unified School District, San Fra...",http://ap.rdcpix.com/2c8996e68c6ec75a52e6496b0...,http://ap.rdcpix.com/2c8996e68c6ec75a52e6496b0...
2,https://www.realtor.com/rentals/details/570-Al...,2242564289,2974926737,MRCA,LG24233049,FOR_RENT,Nestled within the exclusive gated community o...,SINGLE_FAMILY,570 Allview Ter,570 Allview Ter,...,NaN,NaN,256084.0,O-MRCA-LCNP,Coldwell Banker Realty - Newport Beach,900624.lead@leads.leadrouter.com,"[{'number': '9496441600', 'type': 'Office', 'p...",Laguna Beach Unified School District,http://ap.rdcpix.com/6fc02ad4056a0ddd6d8e730e8...,http://ap.rdcpix.com/6fc02ad4056a0ddd6d8e730e8...
3,https://www.realtor.com/rentals/details/11059-...,2254354449,2966780956,ZILL,43vhmf4dy544d,FOR_RENT,LUXURY APARTMENTS THAT SUIT YOUR LIFESTYLE Loc...,APARTMENT,11059 McCormick St,11059 McCormick St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles Unified School District,http://ar.rdcpix.com/a86f04bd532ff0ce43250358b...,http://ar.rdcpix.com/a86f04bd532ff0ce43250358b...
4,https://www.realtor.com/rentals/details/450-S-...,9608413152,2951082408,APTL,p31967036,FOR_RENT,Dare to have it all in a community centered ar...,APARTMENT,450 S La Fayette Park Pl,450 S La Fayette Park Pl,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles Unified School District,http://ar.rdcpix.com/1dee0ba6be9ad1938e786145f...,http://ar.rdcpix.com/1dee0ba6be9ad1938e786145f...


### Drop columns with excessive missing values (>70%)


In [ ]:
missing_values = data.isnull().sum() / len(data) * 100
columns_to_drop = missing_values[missing_values > 70].index.tolist()
data_cleaned = data.drop(columns=columns_to_drop)

### Convert datetime columns to numerical features


In [ ]:
for column in data_cleaned.select_dtypes(include=['datetime64', 'object']).columns:
    try:
        data_cleaned[column] = pd.to_datetime(data_cleaned[column], errors='coerce').map(lambda x: x.timestamp() if pd.notnull(x) else 0)
    except Exception as e:
        print(f'Error converting column {column}: {e}')

<ipython-input-29-e00f9266a7ed>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_cleaned[column] = pd.to_datetime(data_cleaned[column], errors='coerce').map(lambda x: x.timestamp() if pd.notnull(x) else 0)
<ipython-input-29-e00f9266a7ed>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_cleaned[column] = pd.to_datetime(data_cleaned[column], errors='coerce').map(lambda x: x.timestamp() if pd.notnull(x) else 0)
<ipython-input-29-e00f9266a7ed>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_cleaned[column] = pd.to_datetime(data_cleaned[column], errors='coerce').

### One-Hot Encoding to handle categorical features


In [ ]:
data_cleaned = pd.get_dummies(data_cleaned, drop_first=True)


### Handle missing values using SimpleImputer


In [ ]:
imputer = SimpleImputer(strategy='median')
data_cleaned_array = imputer.fit_transform(data_cleaned)
data_cleaned = pd.DataFrame(data_cleaned_array, columns=data_cleaned.columns)


### Ensure all features are numerical


In [ ]:
data_cleaned = data_cleaned.apply(pd.to_numeric, errors='coerce')


### Feature Importance using SelectKBest


In [ ]:
target_variable = 'list_price'
features = data_cleaned.drop(columns=[target_variable])
selector = SelectKBest(score_func=f_regression, k=10)
features_reduced = selector.fit_transform(features, data_cleaned[target_variable])
selected_columns = features.columns[selector.get_support()]

### Retain original features for comparison


In [ ]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(features, data_cleaned[target_variable], test_size=0.2, random_state=42)


### Apply PCA to find a latent manifold


In [ ]:
pca = PCA(n_components=1)
latent_feature = pca.fit_transform(features)
data_cleaned['latent_feature'] = latent_feature

### Amalgamate latent feature with reduced features


In [ ]:
features_reduced = np.hstack([features_reduced, latent_feature])


### Split the data into training and testing sets


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_reduced, data_cleaned[target_variable], test_size=0.2, random_state=42)


### Initialize different regression models


In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=50, max_depth=10),
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'XGBoost': XGBRegressor(n_estimators=50, max_depth=5),
    'MLP Regressor': MLPRegressor(max_iter=200)
}

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
results = {}
for model_name, model in models.items():
    model.fit(X_train_orig, y_train_orig)
    y_pred_orig = model.predict(X_test_orig)
    mse_orig = mean_squared_error(y_test_orig, y_pred_orig)
    r2_orig = r2_score(y_test_orig, y_pred_orig)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[model_name] = {'MSE Before': mse_orig, 'R2 Before': r2_orig, 'MSE After': mse, 'R2 After': r2}


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Display results in a table


In [ ]:
results_df = pd.DataFrame(results).T
print(results_df)

                     MSE Before      R2 Before     MSE After      R2 After
Linear Regression  2.725811e+07       0.762778  3.156411e+07      0.725303
Random Forest      8.478899e+06       0.926210  3.796656e+07      0.669584
KNN                2.601931e+08      -1.264414  2.603022e+08     -1.265363
XGBoost            2.623145e+07       0.771712  1.818965e+08     -0.583013
MLP Regressor      1.395479e+13 -121445.037571  8.051612e+12 -70070.736626


### Implementing a Multi-Layer Perceptron using Keras


In [ ]:
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))





Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 30525829368774656.0000 - mae: 128455400.0000 - val_loss: 1167314934300672.0000 - val_mae: 26697682.0000
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 834861178814464.0000 - mae: 20981024.0000 - val_loss: 310828226052096.0000 - val_mae: 12546921.0000
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 249026716893184.0000 - mae: 10206323.0000 - val_loss: 122334375575552.0000 - val_mae: 6095735.5000
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 70671019278336.0000 - mae: 5094158.0000 - val_loss: 76339520471040.0000 - val_mae: 4495248.5000
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 53937671503872.0000 - mae: 4363988.5000 - val_loss: 60247787962368.0000 - val_mae: 4151937.5000
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 46681655607296.0000 - mae: 4561861.5000 - val_loss: 47095805378560.0000 - val_mae: 3480659.0000
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 306222318

### Make predictions


In [ ]:

y_pred_mlp = model.predict(X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


### Evaluate the model


In [ ]:
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)
print(f'MLP Mean Squared Error: {mse_mlp}')
print(f'MLP R2 Score: {r2_mlp}')

MLP Mean Squared Error: 18496684714520.688
MLP R2 Score: -160972.34044591206


### Adding MLP results to the comparison table


In [ ]:

results['MLP Keras'] = {'MSE': mse_mlp, 'R2': r2_mlp}
results_df = pd.DataFrame(results).T
print(results_df)



                            MSE             R2
Linear Regression  3.161373e+07       0.724871
Random Forest      4.229384e+07       0.631924
KNN                2.716781e+08      -1.364366
XGBoost            1.796988e+08      -0.563886
MLP Regressor      6.395782e+12  -55660.346356
MLP Keras          1.849668e+13 -160972.340446


### Clear memory to avoid crashes


In [ ]:
gc.collect()

156